In [1]:
import sys
import os
import csv
import random
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from force_utils import DataProcessing

Using TensorFlow backend.


In [2]:
# -------------------------------------------------------------------------------------
# DATASET
# -------------------------------------------------------------------------------------

# Input
x_train = np.load('./dataset/features/dense_train.npy')
x_test = np.load('./dataset/features/dense_val.npy')

dataset = DataProcessing('./dataset/video.mp4', './dataset/forceinfo.csv', p=True, f=True, name=False)

# Split train and validation
keys = list(dataset.f_dict.keys())
values = list(dataset.f_dict.values())
train_num = int(round(0.8 * len(keys)))

In [3]:
print(x_train.shape, x_test.shape, len(keys), len(values), train_num)

(45280, 9, 9, 1024) (11296, 9, 9, 1024) 56602 56602 45282


In [4]:
y_train = values[:train_num-2]
y_test = values[train_num:-24]

In [ ]:
# Target
y_train = values[:train_num]
for x in values[:30]:
    y_train.append(x)
#y_train = np.array(y_train) / np.max(values)

y_test = values[train_num:]
for x in values[train_num:][:8]:
    y_test.append(x)
#y_test = np.array(y_test) / np.max(values)

In [5]:
# -------------------------------------------------------------------------------------
# VARIABLE
# -------------------------------------------------------------------------------------

input_shape = (299, 299, 3)
batch_size = 32
epochs = 100
learning_rate = 0.01

In [6]:
# -------------------------------------------------------------------------------------
# MODEL
# -------------------------------------------------------------------------------------

input_tensor = Input(shape=x_train.shape[1:])
x = Flatten(input_shape=x_train.shape[1:])(input_tensor)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(1)(x)
model = Model(inputs=input_tensor, outputs=prediction)

In [7]:
# -------------------------------------------------------------------------------------
# COMPILE
# -------------------------------------------------------------------------------------

# Optimizer
optim = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

# Learning rate scheduler
def schedule(epoch, decay=0.9):
    return learning_rate * decay**(epoch)

# Callbacks
callbacks = [
    ModelCheckpoint('dataset/weights/dense_{epoch:02d}.h5', monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')]
    # LearningRateScheduler(schedule)]

# Compile
model.compile(optimizer=optim, loss='mean_absolute_error')

In [ ]:
# -------------------------------------------------------------------------------------
# FIT
# -------------------------------------------------------------------------------------

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)

save_history(history, 'dataset/history/dense.txt')

Train on 45280 samples, validate on 11296 samples
Epoch 1/100
45280/45280 [==============================] - 40s 885us/step - loss: 1.4815 - val_loss: 0.4126

Epoch 00001: val_loss improved from inf to 0.41256, saving model to dataset/weights/dense_01.h5
Epoch 2/100
45280/45280 [==============================] - 28s 628us/step - loss: 0.3541 - val_loss: 0.4440

Epoch 00002: val_loss did not improve from 0.41256
Epoch 3/100
45280/45280 [==============================] - 30s 659us/step - loss: 0.3240 - val_loss: 0.4094

Epoch 00003: val_loss improved from 0.41256 to 0.40940, saving model to dataset/weights/dense_03.h5
Epoch 4/100
45280/45280 [==============================] - 29s 631us/step - loss: 0.2976 - val_loss: 0.3955

Epoch 00004: val_loss improved from 0.40940 to 0.39549, saving model to dataset/weights/dense_04.h5
Epoch 5/100
45280/45280 [==============================] - 30s 661us/step - loss: 0.2789 - val_loss: 0.4249

Epoch 00005: val_loss did not improve from 0.39549
Epoch 6